In [ ]:
class ResSaNet50(nn.Module):
  def __init__(self, dim_conv_1=64, 
               dim_stage_1=64,  
               dim_stage_2=128,
               dim_stage_3=256, 
               dim_stage_4=512, 
               dim_FC=512, num_classes=num_classes
               ):
    super().__init__()

    layers = []
    layers.append(nn.Conv2d(3, dim_conv_1, kernel_size=1, kernel_size=7, stride=2, padding=3))
    for i in range(3):
      layers.append(nn.Conv2d(dim_conv_1, dim_conv_1, kernel_size=3, padding=1))
    layers.append(nn.Conv2d(dim_conv_1, dim_stage_1, kernel_size=2))
    for i in range(3):
      layers.append(IBasic_Block(dim_stage_1, dim_stage_1))
    layers.append(nn.Conv2d(dim_stage_1, dim_stage_2, kernel_size=2))
    for i in range(4):
      layers.append(SE_IBasicBlock(dim_stage_2, dim_stage_2))
    layers.append(nn.Conv2d(dim_stage_2, dim_stage_3, kernel_size=2))
    for i in range(14):
      layers.append(SE_IBasicBlock(dim_stage_3, dim_stage_3))
    layers.append(nn.Conv2d(dim_stage_3, dim_stage_4, kernel_size=2))
    # for i in range(3):
    #   layers.append(IBasic_Transformer(dim_in_stage_4, dim_out_stage_4))
    # layers.append(nn.Linear(dim_in_FC, num_classes))

    self.main = nn.Sequential(*layers)

    k=1
    self.MBConv = nn.Sequential(
      nn.Conv2d(dim_stage_4, dim_stage_4, kernel_size=1, stride=1, padding=1),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(dim_stage_4, affine=True, track_running_stats=True),
      nn.Conv2d(dim_stage_4, k*dim_stage_4, kernel_size=3, stride=1, padding=1, groups=dim_stage_4),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(dim_stage_4, affine=True, track_running_stats=True),
     # SE_block(dim_stage_4, dim_stage_4, stride=1, padding=1, enable=True),
      nn.Conv2d(dim_stage_4, dim_stage_4, kernel_size=1, stride=1, padding=1),
      nn.BatchNorm2d(dim_stage_4, affine=True, track_running_stats=True))

    self.IBSA = IBSA_Block(dim_stage_4, dim_stage_4)
    self.conv = nn.Conv2d(dim_stage_4, dim_FC, kernel_size=1)
    self.fc = nn.Linear(dim_FC, num_classes)

  def forward(self, x):
    output = self.main(x)
    output = self.IBSA(output)
    output += self.MBConv(output)
    output = self.fc(self.conv(output))
    return output